<a href="https://colab.research.google.com/github/Rajnandanigithub/Classifying_documents/blob/main/Classifying_Documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Necessary Libraries

In [5]:
# # All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented.
# !pip install -qy pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 scikit-learn==0.20.1
# # - Update a specific package
# !pip install pmdarima -U
# # - Update a package to specific version
# !pip install --upgrade pmdarima==2.0.2
# # Note: If your environment doesn't support "!pip install", use "!mamba install"

In [6]:
# !pip install -Uqq portalocker>=2.0.0
# !pip install -qq torchtext
# !pip install -qq torchdata
# !pip install -Uqq plotly
# !pip install -qq dash

In [31]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from itertools import accumulate
import matplotlib.pyplot as plt
from torchtext.data.utils import get_tokenizer

import torch
import torch.nn as nn

from torch.utils.data import DataLoader
import numpy as np
from torchtext.datasets import AG_NEWS
from IPython.display import Markdown as md
from tqdm import tqdm

from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import AG_NEWS
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from sklearn.manifold import TSNE
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split

from torchtext.data.utils import get_tokenizer

# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [32]:
import torch
print(torch.__version__)

2.1.0+cu121


In [33]:
import torchtext
print(torchtext.__version__)

0.16.0+cpu


In [10]:
# !pip show torchtext

In [11]:
# !pip show torch

helper Function

In [34]:
def plot(COST,ACC):
    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.plot(COST, color=color)
    ax1.set_xlabel('epoch', color=color)
    ax1.set_ylabel('total loss', color=color)
    ax1.tick_params(axis='y', color=color)

    ax2 = ax1.twinx()
    color = 'tab:blue'
    ax2.set_ylabel('accuracy', color=color)  # you already handled the x-label with ax1
    ax2.plot(ACC, color=color)
    ax2.tick_params(axis='y', color=color)
    fig.tight_layout()  # otherwise the right y-label is slightly clipped

    plt.show()

Text Classification

Building A text classification model using PyTorch and torchtext to classify news articles into one of the four categories: World, Sports, Business, and Sci/Tech.

### Import  dataset

Load the AG_NEWS dataset for the train split and split it into input text and corresponding labels:


In [35]:
train_iter= iter(AG_NEWS(split="train"))

The AG_NEWS dataset in torchtext does not support direct indexing like a list or tuple. It is not a random access dataset but rather an iterable dataset that needs to be used with an iterator. This approach is more effective for text data.


In [36]:
y,text= next((train_iter))
print(y)
print(text)

3
Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.


In [37]:
x,text= next((train_iter))
print(y)
print(text)

3
Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.


In [38]:
ag_news_label = {1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tec"}
ag_news_label[y]

'Business'

In [25]:
ag_news_label = {1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tec"}
ag_news_label[x]

'Business'

In [27]:
## to look at the data we can convert the iterable data into dataframe

In [39]:
data = []
for label, text in train_iter:
    data.append([label, text])

# Create a pandas DataFrame from the list
df = pd.DataFrame(data, columns=["Label", "Text"])

# Display the first few rows of the DataFrame
df.head()

,Label,Text
0,3,Oil and Economy Cloud Stocks' Outlook (Reuters...
1,3,Iraq Halts Oil Exports from Main Southern Pipe...
2,3,"Oil prices soar to all-time record, posing new..."
3,3,"Stocks End Up, But Near Year Lows (Reuters) Re..."
4,3,Money Funds Fell in Latest Week (AP) AP - Asse...


In [40]:
## check total labels
df["Label"].unique()

array([3, 4, 2, 1])

creating tokens

In [41]:
# Reinitialize train_iter
train_iter = AG_NEWS(split="train")

# Define tokenizer and yield_tokens
tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text.lower())  # Lowercase conversion for consistency

# Build vocabulary
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Print the vocabulary size and sample tokens
print(f"Vocabulary size: {len(vocab)}")
print(f"Sample tokens: {list(vocab.get_stoi().keys())[:10]}")


Vocabulary size: 95811
Sample tokens: ['zzz', 'zygmunt', 'zwiki', 'zvidauri', 'zurine', 'zurab', 'zuo', 'zuloaga', 'zovko', 'zotinca']


In [42]:
vocab(["age","hello"])

[2120, 12544]

In [43]:
# Split the dataset into training and testing iterators.
train_iter, test_iter = AG_NEWS()

# Convert the training and testing iterators to map-style datasets.
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

# Determine the number of samples to be used for training and validation (5% for validation).
num_train = int(len(train_dataset) * 0.95)

# Randomly split the training dataset into training and validation datasets using `random_split`.
# The training dataset will contain 95% of the samples, and the validation dataset will contain the remaining 5%.
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [45]:
def text_pipeline(x):
  return vocab(tokenizer(x))

def label_pipeline(x):
   return int(x) - 1

In [46]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [47]:
BATCH_SIZE = 64

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)

In [48]:
label, text, offsets=next(iter(valid_dataloader ))
label, text, offsets

(tensor([2, 3, 3, 1, 1, 0, 2, 3, 3, 1, 0, 3, 3, 0, 2, 2, 2, 2, 2, 3, 3, 0, 2, 1,
         2, 3, 1, 3, 0, 0, 1, 0, 3, 3, 0, 0, 1, 3, 0, 2, 1, 3, 2, 2, 3, 2, 2, 2,
         3, 2, 2, 2, 2, 1, 1, 0, 0, 1, 3, 2, 1, 3, 3, 0]),
 tensor([943,  17, 193,  ..., 273, 647,   1]),
 tensor([   0,   29,   59,   97,  125,  167,  211,  250,  294,  339,  375,  420,
          466,  497,  548,  582,  630,  677,  729,  776,  822,  865,  926, 1018,
         1063, 1099, 1133, 1180, 1214, 1250, 1278, 1317, 1369, 1412, 1454, 1537,
         1560, 1602, 1647, 1677, 1718, 1762, 1805, 1839, 1883, 1925, 1972, 2033,
         2073, 2095, 2167, 2210, 2254, 2302, 2348, 2387, 2422, 2472, 2524, 2569,
         2603, 2635, 2687, 2715]))

In [49]:
from torch import nn

class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [50]:
emsize=64

In [51]:
vocab_size=len(vocab)
vocab_size

95811

In [54]:
model = TextClassificationModel(vocab_size, emsize, num_class=4).to(device)
model

TextClassificationModel(
  (embedding): EmbeddingBag(95811, 64, mode='mean')
  (fc): Linear(in_features=64, out_features=4, bias=True)
)

In [55]:
predicted_label=model(text, offsets)

In [56]:
predicted_label.shape

torch.Size([64, 4])

In [58]:
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return ag_news_label[output.argmax(1).item() + 1]

In [59]:
predict("I like sports",text_pipeline )

'Sci/Tec'

In [60]:
#Create a function to evaluate the model's accuracy on a dataset.


In [61]:
def evaluate(dataloader):
    model.eval()
    total_acc, total_count= 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)

            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [62]:
evaluate(test_dataloader)

0.2143421052631579